In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras.layers import BatchNormalization, Conv2D, Activation , AveragePooling2D , Input ,Dropout
from tensorflow.keras.layers import Dense,  MaxPooling2D, Add, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import PReLU


In [5]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0,1], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0,1],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

TypeError: list indices must be integers or slices, not tuple

In [2]:
# 학습 데이터 준비 
path = '/home/lab07/landmark/'


train_path = '/home/lab07/landmark/landmark_image'

img_dirs = os.listdir(train_path) # 해당 path안에 폴더 이름 리스트에 담기
label_df = pd.read_csv(path + '/category.csv') # 각 랜드마크별 label

label_dict = dict(label_df[['landmark_name', 'landmark_id']].values) # 폴더경로 저장용
label_dict_reverse = dict(label_df.values) # 시각화용

In [3]:
# 파일(JPG)명과 label 정보를 담은 데이터 프레임 생성
train_files = []
train_categories=[]
for img_dir in img_dirs:
    img_dir_list = os.listdir(train_path + '/' + img_dir)
    for filename in img_dir_list:
        train_files.append(img_dir + '/' + filename)
        train_categories.append(label_dict[img_dir])
    
train_data=pd.DataFrame(
                    {"file":train_files,
                    "label":train_categories}
                )    

train_data.head()

,file,label
0,수영구청/수영구청_011.JPG,794
1,수영구청/수영구청_008.JPG,794
2,수영구청/수영구청_060.JPG,794
3,수영구청/수영구청_075.JPG,794
4,수영구청/수영구청_023.JPG,794


In [4]:
train_df, valid_df = train_test_split(train_data, test_size=0.2, random_state=42,
                stratify=train_data['label'])

In [5]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                    vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    train_path,
    x_col= "file",
    y_col= "label",
    target_size = (400,400),
    class_mode = "raw",
    batch_size = 12
)


Found 70481 validated image filenames.


In [6]:

validation_generator = validation_datagen.flow_from_dataframe(
    valid_df,
    train_path,
    x_col= "file",
    y_col= "label",
    target_size = (400,400),
    class_mode = "raw",
    batch_size = 12
)

Found 17621 validated image filenames.


In [8]:

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow_addons.optimizers import RectifiedAdam
from tensorflow.keras.optimizers import Adam,SGD
# opt = tf.keras.optimizers.Adam(learning_rate = LR)


strategy = tf.distribute.MirroredStrategy()

with strategy.scope():

# def scheduler(epoch):
#   if epoch < 10:
#     return 0.0001
#   else:
#     return 0.0001 * tf.math.exp(0.1 * (10 - epoch))
# with tf.device('/device:GPU:0'):

    def get_lr_callback():
        lr_start   = 0.000001*10*0.5
        lr_max     = 0.0000005 * 12 * 10*0.5
        lr_min     = 0.000001 * 10*0.5
        lr_ramp_ep = 5
        lr_sus_ep  = 0
        lr_decay   = 0.8


        def lrfn(epoch):
            if epoch < lr_ramp_ep:
                lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start   
            elif epoch < lr_ramp_ep + lr_sus_ep:
                lr = lr_max    
            else:
                lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
            return lr

        lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = False)
        return lr_callback

    # learning = tf.keras.callbacks.LearningRateScheduler(get_lr_callback)



    DenseNet201_base = DenseNet201(weights='imagenet',
                              include_top=False,
                              input_shape=(400,400,3))

    DenseNet201_base.trainable = True  # Convolution Layer 동결



    model = Sequential()


    model.add(DenseNet201_base)


    model.add(GlobalAveragePooling2D())

    model.add(Dense(1049,
                    activation='softmax'))

    model.summary()

    # model.load_weights('./project_model/efficientnetb6.h5')

    # Model saving callback
    checkpointer = ModelCheckpoint(filepath='./dense_400_400.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True)


    # opt = tf.keras.optimizers.Adam(learning_rate = LR)


    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', verbose=2, patience=5)

    model.compile(optimizer=RectifiedAdam(learning_rate=1e-4),
                  loss='sparse_categorical_crossentropy',
                  metrics=['sparse_categorical_accuracy'])

    history = model.fit(train_generator,
                        steps_per_epoch=len(train_df)/12, # batch_size * steps_per_epoch > data의 양 xxxxx
                        epochs=10000,
                        callbacks=[checkpointer, early_stopping, get_lr_callback()],
                        validation_data=validation_generator,
                        validation_steps=len(valid_df)/12)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensor

KeyboardInterrupt: 